## Reading files

### Official Doc

https://pandas.pydata.org/pandas-docs/stable/

    
5 useful "read_csv" parameters that are often overlooked:
    
+ names: specify column names
+ usecols: which columns to keep
+ dtype: specify data types
+ nrows: # of rows to read
+ na_values: strings to recognize as NaN

In [2]:
import pandas as pd

In [4]:
CSV = 'data/skip_rows.csv'
pd.read_csv(CSV, header=2, skiprows=[3,4], sep=';')

,A,B,C
0,10,20,30
1,40,50,60
2,70,80,90


Two easy ways to reduce DataFrame memory usage:
1. Only read in columns you need
2. Use 'category' data type with categorical data.

In [15]:
CSV_CAT = 'data/category_cols.csv'

df = pd.read_csv(CSV_CAT, usecols=['City','Data','Country'], dtype={'Country':'category'})
df.head()

,City,Data,Country
0,Madrid,20,Spain
1,Paris,50,France
2,Cadiz,80,Spain


In [16]:
df.dtypes

City         object
Data          int64
Country    category
dtype: object

You can read directly from a compressed file:
`df = pd.read_csv(http://'https://t.co/3JAwA8h7FJ')`

Or write to a compressed file:
`df.to_csv(http://'https://t.co/3JAwA8h7FJ')`

Also supported: .gz, .bz2, .xz

Do you sometimes end up with an "Unnamed: 0" column in your DataFrame? 🤔

Solution: Set the first column as the index (when reading)
Alternative: Don't save the index to the file (when writing)

In [26]:
TEMP_CSV = 'data/temp_csv.csv'
df = pd.DataFrame({'A':[1,2,3], 'B':[5,6,7]})
df

,A,B
0,1,5
1,2,6
2,3,7


In [27]:
df.to_csv(TEMP_CSV)
df = pd.read_csv(TEMP_CSV)
df

,Unnamed: 0,A,B
0,0,1,5
1,1,2,6
2,2,3,7


In [28]:
df = pd.read_csv(TEMP_CSV, index_col=0)
df

,A,B
0,1,5
1,2,6
2,3,7


Are your dataset rows spread across multiple files, but you need a single DataFrame?

Solution:
1. Use glob() to list your files
2. Use a generator expression to read files and concat() to combine them

In [38]:
from glob import glob

GLOB_SEARCH = "data/ex_glob_datasets/glob_ex_*.csv"
sorted(glob(GLOB_SEARCH))

['data/ex_glob_datasets\\glob_ex_1.csv',
 'data/ex_glob_datasets\\glob_ex_2.csv',
 'data/ex_glob_datasets\\glob_ex_3.csv']

In [39]:
pd.concat((pd.read_csv(file) for file in sorted(glob(GLOB_SEARCH))), ignore_index=True)

,A,B,C
0,1,2,3
1,4,5,6
2,7,8,9
3,55,66,77
4,44,88,99
5,444,555,666
6,777,888,999


Do you need to build a DataFrame from multiple files, but also keep track of which row came from which file?

1. List files w/ glob()
2. Read files w/ gen expression, create new column w/ assign(), combine w/ concat()

In [40]:
from glob import glob

GLOB_SEARCH = "data/ex_glob_datasets/glob_ex_*.csv"
sorted(glob(GLOB_SEARCH))

['data/ex_glob_datasets\\glob_ex_1.csv',
 'data/ex_glob_datasets\\glob_ex_2.csv',
 'data/ex_glob_datasets\\glob_ex_3.csv']

In [43]:
pd.concat((pd.read_csv(file).assign(filename=file) for file in sorted(glob(GLOB_SEARCH))), ignore_index=True)

,A,B,C,filename
0,1,2,3,data/ex_glob_datasets\glob_ex_1.csv
1,4,5,6,data/ex_glob_datasets\glob_ex_1.csv
2,7,8,9,data/ex_glob_datasets\glob_ex_1.csv
3,55,66,77,data/ex_glob_datasets\glob_ex_2.csv
4,44,88,99,data/ex_glob_datasets\glob_ex_2.csv
5,444,555,666,data/ex_glob_datasets\glob_ex_3.csv
6,777,888,999,data/ex_glob_datasets\glob_ex_3.csv


Want to read a HUGE dataset into pandas but don't have enough memory?

Randomly sample the dataset *during file reading* by passing a function to "skiprows"